"""The file creates .db from .xlsx tables taken from https://data.mos.ru/opendata
"""

In [1]:
import pandas as pd
import re
import sqlite3 as sql

In [2]:
df_c = pd.read_excel("./raw_data/rest.xlsx")[["TypeObject", "Name", "Latitude_WGS84", "Longitude_WGS84", "Address", "PublicPhone"]]
df_c.head()

,TypeObject,Name,Latitude_WGS84,Longitude_WGS84,Address,PublicPhone
0,кафе,СМЕТАНА,55.879002,37.714565,"город Москва, улица Егора Абакумова, дом 9",(499) 183-14-10
1,кафе,Родник,55.738239,37.673306,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35
2,кафе,Кафе «Академия»,55.735511,37.669648,"город Москва, Абельмановская улица, дом 6",(926) 841-45-78
3,кафе,ПИЦЦЕТОРИЯ,55.891957,37.573329,"город Москва, Абрамцевская улица, дом 1",(499) 200-17-09
4,кафе,Кафе «Вишневая метель»,55.904086,37.572306,"город Москва, Абрамцевская улица, дом 9, корпус 1",(499) 200-00-22


In [3]:
df_m = pd.read_excel("./raw_data/metro.xlsx").groupby(by="NameOfStation")["Latitude_WGS84", "Longitude_WGS84"].mean()
df_m["TypeObject"] = "метро"
df_m.rename_axis("Name", inplace=True)
df_m.reset_index(inplace=True)
df_m['Name'] = df_m['Name'].str.lower()
df_m.head()

,Name,Latitude_WGS84,Longitude_WGS84,TypeObject
0,авиамоторная,55.751519,37.717090,метро
1,автозаводская,55.706863,37.658926,метро
2,академическая,55.687878,37.573643,метро
3,александровский сад,55.752240,37.610630,метро
4,алексеевская,55.807824,37.638535,метро


In [4]:
df_z = pd.read_excel("./raw_data/zags.xlsx")
df_z = df_z[["CommonName", "geoData", "Address", "ContactPhone"]]
df_z["type"] = "загс"
df_z_c = df_z["geoData"].apply(lambda x: re.findall(r"\d+[\,\.]\d+", x))
df_z[["longitude", "latitude"]] = pd.DataFrame(df_z_c.values.tolist())
for c in ["longitude", "latitude"]:
    df_z[c] = pd.to_numeric(df_z[c])
df_z.drop("geoData", axis=1, inplace=True)
df_z.rename({"CommonName": "name", "Address": "address",
             "ContactPhone": "phone"}, axis='columns', inplace=True)
df_z.head()

,name,address,phone,type,longitude,latitude
0,Кутузовский ЗАГС,"город Москва, Кутузовский проспект, дом 23, ко...",(495) 777-77-77,загс,37.548325,55.744518
1,Левобережный ЗАГС,"город Москва, Фестивальная улица, дом 9",(495) 777-77-77,загс,37.474323,55.856744
2,Люблинский ЗАГС,"город Москва, 8-я улица Текстильщиков, дом 14",(495) 777-77-77,загс,37.746249,55.701949
3,Медведковский ЗАГС,"город Москва, улица Молодцова, дом 1А",(495) 777-77-77,загс,37.646247,55.876957
4,Мещанский ЗАГС,"город Москва, проспект Мира, дом 16, строение 1",(495) 777-77-77,загс,37.632893,55.775698


In [5]:
df = pd.concat([df_c, df_m], ignore_index=True)

df.rename({"TypeObject": "type", "Name": "name",
           "Latitude_WGS84": "latitude", "Longitude_WGS84": "longitude",
           "Address": "address", "PublicPhone": "phone"},
          axis='columns', inplace=True)
df = pd.concat([df, df_z], ignore_index=True)
to_rename = {"магазин (отдел кулинарии)": "кулинария",
             "предприятие быстрого обслуживания": "фастфуд"}
df["type"] = df["type"].apply(lambda x: x if x not in to_rename else to_rename[x])
df = df[["type", "name", "latitude", "longitude", "address", "phone"]]
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,type,name,latitude,longitude,address,phone
0,кафе,СМЕТАНА,55.879002,37.714565,"город Москва, улица Егора Абакумова, дом 9",(499) 183-14-10
1,кафе,Родник,55.738239,37.673306,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35
2,кафе,Кафе «Академия»,55.735511,37.669648,"город Москва, Абельмановская улица, дом 6",(926) 841-45-78
3,кафе,ПИЦЦЕТОРИЯ,55.891957,37.573329,"город Москва, Абрамцевская улица, дом 1",(499) 200-17-09
4,кафе,Кафе «Вишневая метель»,55.904086,37.572306,"город Москва, Абрамцевская улица, дом 9, корпус 1",(499) 200-00-22


In [6]:
try:
    conn = sql.connect("./data/data.db")
    df.to_sql('places', con=conn)    
except sql.Error as e:
    print(e)
finally:
    conn.close()